# Azure AI Agents s podporou Model Context Protocol (MCP)

Tento notebook ukazuje, jak používat Azure AI Agents s nástroji Model Context Protocol (MCP) k vytvoření inteligentního agenta, který může využívat externí MCP servery pro rozšířené schopnosti.


## Instalace požadovaných balíčků NuGet

Nejprve je potřeba nainstalovat balíček Azure AI Agents Persistent, který poskytuje základní funkce pro práci s Azure AI Agents.


## Výhody autentizace bez klíčů

Tento notebook demonstruje **autentizaci bez klíčů**, která přináší několik výhod:
- ✅ **Žádné API klíče k spravování** - Využívá autentizaci založenou na identitě Azure
- ✅ **Zvýšená bezpečnost** - Žádná tajemství uložená v kódu nebo konfiguraci
- ✅ **Automatická rotace přihlašovacích údajů** - Azure se stará o životní cyklus přihlašovacích údajů
- ✅ **Přístup založený na rolích** - Využívá Azure RBAC pro detailní nastavení oprávnění

`DefaultAzureCredential` automaticky použije nejlepší dostupný zdroj přihlašovacích údajů:
1. Spravovaná identita (při běhu v Azure)
2. Přihlašovací údaje Azure CLI (během vývoje)
3. Přihlašovací údaje Visual Studio
4. Proměnné prostředí (pokud jsou nastaveny)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Nainstalujte balíček Azure Identity pro ověřování pomocí služeb Azure pomocí DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importujte požadované obory názvů

Importujte potřebné obory názvů pro Azure AI Agents a Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurace klienta Azure AI Agent (autentizace bez klíče)

Nastavte konfigurační proměnné a vytvořte PersistentAgentsClient pomocí **autentizace bez klíče**:
- **projectEndpoint**: Koncový bod projektu Azure AI Foundry
- **modelDeploymentName**: Název nasazeného AI modelu (GPT-4.1 nano)
- **mcpServerUrl**: URL serveru MCP (Microsoft Learn API)
- **mcpServerLabel**: Štítek pro identifikaci serveru MCP
- **DefaultAzureCredential**: Používá spravovanou identitu, Azure CLI nebo jiné zdroje přihlašovacích údajů (nejsou vyžadovány API klíče)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Vytvoření definice nástroje MCP

Vytvořte definici nástroje MCP, který se připojí k serveru Microsoft Learn MCP. To umožní agentovi přístup k obsahu a dokumentaci Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Vytvoření AI agenta

Vytvořte trvalého AI agenta s určeným modelem a nástroji MCP. Agent je nakonfigurován s:
- Modelem GPT-4.1 nano
- Pokyny k použití nástrojů MCP pro asistenci
- Přístupem k serveru Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Vytvoření vlákna a odeslání zprávy

Vytvořte konverzační vlákno a pošlete uživatelskou zprávu s dotazem na rozdíl mezi Azure OpenAI a OpenAI. Tím otestujete schopnost agenta používat nástroje MCP k poskytování přesných informací.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Nastavení zdrojů nástroje MCP (bez klíče)

Nastavte zdroje nástroje MCP. Pro skutečně bezklíčový přístup můžete odstranit vlastní hlavičky, pokud server MCP podporuje ověřování na základě identity Azure. Níže uvedený příklad ukazuje, jak přidat hlavičky, pokud je to potřeba, ale pro bezklíčové scénáře nemusí být vyžadovány.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Spuštění agenta

Vytvořte a spusťte proces pro zpracování zprávy uživatele. Agent použije nakonfigurované MCP nástroje a zdroje k vytvoření odpovědi.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitorování spuštění a schvalování nástrojů (bez klíčů)

Sledujte stav spuštění agenta a zpracovávejte potřebná schválení nástrojů. Tento cyklus:
1. Čeká na dokončení spuštění nebo na vyžádání akce
2. Automaticky schvaluje volání nástrojů MCP, pokud je to potřeba
3. U autentizace bez klíčů nemusí být hlavičky nutné, pokud server MCP podporuje identitu Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Zobrazení výsledků konverzace

Získejte a zobrazte všechny zprávy v vlákně, včetně otázky uživatele a odpovědi agenta. Zprávy jsou zobrazeny v chronologickém pořadí s časovými značkami a indikátory rolí.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
